In [1]:
install.packages(c("shiny","shinythemes"))

Installing packages into 'C:/Users/AliHashmi/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)



package 'shiny' successfully unpacked and MD5 sums checked
package 'shinythemes' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\AliHashmi\AppData\Local\Temp\RtmpGYEqPT\downloaded_packages


In [15]:
install.packages("janitor")

Installing package into 'C:/Users/AliHashmi/AppData/Local/R/win-library/4.2'
(as 'lib' is unspecified)

also installing the dependency 'snakecase'




package 'snakecase' successfully unpacked and MD5 sums checked
package 'janitor' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\AliHashmi\AppData\Local\Temp\RtmpGYEqPT\downloaded_packages


In [2]:
getwd()

[1] "C:/Users/AliHashmi/Desktop/Development/nutmeghealth/notebooks"

# Example: A sample Shiny App to view CMS Healthcare Data

By Steve Sanderson

Adapted from <https://www.spsanderson.com/steveondata/posts/rtip-2023-04-03/>

In [5]:
project_root <- 'C:/Users/AliHashmi/Desktop/Development/nutmeghealth'
data_dir <- file.path(project_root, 'DATA', 'prov')

In [12]:
CMS_PROV_ROOT <- 'https://data.cms.gov/provider-data/sites/default/files/archive'
prov_data_url <- paste(CMS_PROV_ROOT, '/Hospitals/current/hospitals_current_data.zip', sep='')

In [20]:
current_hosp_data <- function() {

  # URL for file
  # url <- ""

  # Create a temporary directory to process the zip file
  # tmp_dir <- tempdir()
  tmp_dir <- data_dir
  download_location <- file.path(tmp_dir, "download.zip")
  extract_location <- file.path(tmp_dir, "extract")

  # Download the zip file to the temporary location
  utils::download.file(
    url = prov_data_url,
    destfile = download_location
  )

  # Unzip the file
  utils::unzip(download_location, exdir = extract_location)

  # Read the csv files into a list
  csv_file_list <- list.files(
    path = extract_location,
    pattern = "\\.csv$",
    full.names = TRUE
  )

  # make named list
  csv_names <-
    stats::setNames(
      object = csv_file_list,
      nm =
        csv_file_list |>
        basename() |>
        gsub(pattern = "\\.csv$", replacement = "") |>
        janitor::make_clean_names()
    )

  # Process CSV Files
  parse_csv_file <- function(file) {
    # Normalize the path to use C:/path/to/file structure
    normalizePath(file, "/") |>
      # read in the csv file and use check.names = FALSE because some of
      # the names are very long
      utils::read.csv(check.names = FALSE) |>
      dplyr::as_tibble() |>
      # clean the field names
      janitor::clean_names()
  }

  list_of_tables <- lapply(csv_names, parse_csv_file)

  # unlink(tmp_dir, recursive = TRUE)

  # Return the tibbles
  # Add and attribute and a class type to the object
  attr(list_of_tables, ".list_type") <- "current_hosp_data"
  class(list_of_tables) <- c("current_hosp_data", class(list_of_tables))

  list_of_tables
}

In [22]:
hosp_data <- current_hosp_data()